In [2]:
from torch.utils import data
from torch.nn.utils import rnn

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [5]:
import networkx as nx

In [8]:
with open('../Dataset/Cora/cora.cites') as fs:
    for line in fs:
        temp = list(map(int,line.strip().split()))
        print(temp[0],temp[1])
        break

35 1033


In [10]:
# construct network
G = nx.Graph()
count = 0
node_list = {} # assign a id of 0 - n (n nodes...)
with open('../Dataset/Cora/cora.cites') as fs:
    for line in fs:
        u,v = map(int,line.strip().split())
        if u not in node_list:
            node_list[u] = count
            count+=1
        if v not in node_list:
            node_list[v] = count
            count+=1
        G.add_edge(node_list[u],node_list[v])    


In [18]:
# obtaining node features...
feature = [[] for i in range(len(node_list))]
label = {}
with open('../Dataset/Cora/cora.content') as fs:
    for line in fs:
        temp = line.strip().split()
        node = node_list[int(temp[0])]
        label = temp[-1]
        feature[node] = list(map(int,temp[1:-1]))

In [19]:
feature = torch.FloatTensor(feature)

In [20]:
feature.shape

torch.Size([2708, 1433])

In [38]:
class Encoder(nn.Module):
    def __init__(self,feature,graph,input_size,hidden_size,output_size):
        super(Encoder,self).__init__()
        self.feature = feature
        self.graph = graph
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.W_1 = nn.Parameter(torch.rand(input_size,hidden_size))
        self.W_2 = nn.Parameter(torch.rand(input_size,hidden_size))
        
        self.U_1 = nn.Parameter(torch.rand(hidden_size))
        self.U_2 = nn.Parameter(torch.rand(hidden_size))
        
        self.softmax = nn.Softmax(dim=0)

        
    def forward(self,node_sample):
        
        C_n_1_u = torch.matmul(feature,self.W_1)
        
        C_n_2_u = torch.matmul(feature,self.W_2)
        
        s_node_repr = torch.Tensor()
        
        for node in node_sample:
            
            nbr_1 = torch.Tensor()
            nbr_2 = torch.Tensor()
            # obtaining the first hop neighbors
            neighbors_1 = list(nx.neighbors(G,node))
            
            # obtaining the second hop neighbors
            neighbors_2 = []
            for n in neighbors_1:
                nbr_1 = torch.cat((nbr_1,feature[n]),dim=0) # getting the vectors of neighbors
                neighbors_2 += list(nx.neighbors(G,n))
            
            neighbors_2 = list(set(neighbors_2))
            
            for n in neighbors_2:
                nbr_2 = torch.cat((nbr_2,feature[n]),dim=0) # getting vectors of two-hop neighbors
            
            # calculate attention weights for 1-hop neighbors
            
            att_wt_1 = self.softmax(torch.sum(nbr_1*self.U_1).view(-1,1))
            att_wt_2 = self.softmax(torch.sum(nbr_2*self.U_2).view(-1,1))
            
            output = torch.cat((torch.sum(nbr_1,att_wt_1,dim=0),torch.sum(nbr_2,att_wt_2,dim=0)),dim=1)
            
            s_node_repr = torch.cat((s_node_repr,output),dim=0)
            
        return s_node_repr    
            

In [ ]:
def Loss(node_sample,s_node_sepr):
    for 

In [53]:
import random
from collections import Counter

In [79]:
def randomWalk(G,node,walk_length = 3,walk_num=10): # finding list of similar nodes
    sim_nodes = Counter()
    for i in range(walk_num):
        l = 0
        c_node = node
        while(l<walk_length):
            n = random.select(list(nx.neighbors(G,c_node)),1)
            sim_nodes[n]+=1
            c_node = n
            l+=1
    return list(sim_nodes)        

In [55]:
def findSimilarNodes(G):
    similar_nodes = {}
    for node in G.nodes():
        similar_nodes[node] = randomWalk(G,node)
    return similar_nodes 

In [ ]:
def getSamples(G,sample_nodes,neg_sample_size): # generates positive and negative samples for a given batch of nodes
    pos_sample_edges = list(filter(lambda x:x[0] in sample_nodes and x[1] in sample_nodes,all_edges))
    neg_sample_edges = []
    
    i=0
    while i<neg_sample_size:
        u,v = random.sample(sample_nodes,2)
        if (u,v) not in pos_sample_edges and (v,u) not in pos_sample_edges:
            neg_sample_edges.append((u,v))
            i+=1

In [44]:
#def createBatches(Graph = G,batch_size = l):
batch_size = 100
node_list = list(G.nodes())
random.shuffle(node_list)
all_edges = list(G.edges())
neg_sample_size = 
num_batches = int(len(node_list)/batch_size)+1
for i in range(num_batches):
    # get nodes for batches...
    if i<num_batches-1:
        sample_nodes = node_list[i*batch_size:(i+1)*batch_size]
    else:
        sample_nodes = node_list[i*batch_size:]

    # find positive and negative samples for a batch
    pos_sample_edges, neg_sample_edges = getSamples(G,sample_nodes,neg_sample_size)
    
        

In [74]:
samp = [0,2,3,4]
list(filter(lambda x:x[0] in samp and x[1] in samp,list(G.edges())))

[(0, 2), (0, 3), (0, 4)]

In [60]:
edges = list(G.edges())

In [68]:
x = [(2,3),(7,8)]
list(filter(lambda x:(x[0] in samp and x[1] in samp),edges))

[]

In [23]:
x = torch.tensor([1,2,3,4]).view(1,-1)

In [24]:
y = torch.tensor([5,6,7,8]).view(1,-1)

In [27]:
z = torch.cat((x,y),dim=0)

In [35]:
z

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])

In [32]:
z*x

tensor([[ 1,  4,  9, 16],
        [ 5, 12, 21, 32]])

In [33]:
wt = torch.sum(z*x,dim=1).view(-1,1)

In [36]:
torch.sum(z*wt,dim=0)

tensor([380, 480, 580, 680])

In [37]:
z*wt

tensor([[ 30,  60,  90, 120],
        [350, 420, 490, 560]])

In [47]:
x = Counter()
for i in range(9):
    x[i] = 1

In [75]:
x = list(x)

In [77]:
u,v = random.sample(x,2)

In [78]:
u

(2, 3)